# Text summarization using different LLM models and various techniques:-

1> Using Transdormer Pipeline.



## First let's load the data.  We will use research paper about Attentin All you Need.

In [23]:
!pip install PyPDF2
from PyPDF2 import PdfReader
# provide the path of  pdf file/files.
pdfreader = PdfReader('/content/attention_all_you_need.pdf')

In [24]:
# Read content from pdf.

from typing_extensions import Concatenate
# read text from pdf
text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        text += content

In [25]:
text

'Attention Is All You Need\nAshish Vaswani\x03\nGoogle Brain\navaswani@google.comNoam Shazeer\x03\nGoogle Brain\nnoam@google.comNiki Parmar\x03\nGoogle Research\nnikip@google.comJakob Uszkoreit\x03\nGoogle Research\nusz@google.com\nLlion Jones\x03\nGoogle Research\nllion@google.comAidan N. Gomez\x03y\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser\x03\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin\x03z\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requirin

In [26]:
len(text)

32545

### 1> Use PIPELINE from hugginge face to do text summarization.

In [27]:
#Define model - you can any text summarization model here.

!pip install transformers

# #use model from facebook
# from transformers import BartForConditionalGeneration, BartTokenizer

# # Define the model name
# model_name = "facebook/bart-large-cnn"

# # Load the tokenizer and model
# tokenizer = BartTokenizer.from_pretrained(model_name)
# model = BartForConditionalGeneration.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [28]:
#use model from google

# from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# model_name = "google/pegasus-cnn_dailymail"

# # Load the tokenizer and model
# tokenizer = PegasusTokenizer.from_pretrained(model_name)
# model = PegasusForConditionalGeneration.from_pretrained(model_name)

In [34]:
# Another smaller model.
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Define the model name
model_name = "t5-small"

# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [29]:
#Use a pipeline as a high-level helper


# from transformers import pipeline

# pipe = pipeline("summarization", model= model ,tokenizer = tokenizer,framework = 'pt')


# summarized_text = pipe(text,max_length = 2000, min_length = 500, do_sample = False)

## As we have long text input, we will follow below steps.

1> we divide entire dataset into small chunks using RecursiveCharacterTextSplitter from langchain.text_splitter.

2> We pass each chunks to the model and get summary for that particualr chunk.

3> Join the summary results from all those individuak chunk summary and form one summairse text file.

4> Finally, for best summarised version we will use same model agian to woek on this joined text to predict final summary version as output.

In [30]:
# We have too long text so we nedd to divide enitire blog into chunks and then proicessed.
!pip install langchain
!pip install transformers
!pip install -qU langchain-text-splitters

In [49]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Define the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)


# Split the text into chunks
text_chunks = text_splitter.split_text(text)

In [50]:
# Function to summarize a chunk of text
def summarize_chunk(chunk):
    inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding="longest")
    summary_ids = model.generate(inputs.input_ids, max_length=2000, min_length=500, do_sample=False)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Summarize each chunk and concatenate the results
summarized_text = " ".join([summarize_chunk(chunk) for chunk in text_chunks])


In [51]:
print("Joined Summary :- ", summarized_text)

Joined Summary :-  a new single-model state-of-the-art BLEU score of 41.0 after training for 3.5 days on eight GPUs. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results. Our model establishes a new single-model state-of-the-art BLEU score of 41.0 after training for 3.5 days on eight GPUs. a new single-model state-of-the-............................................................................................................................................................................................................................................................................................................................................................ and, and,,, while also improving model performance in case of the latter. The fundamental constraint of sequential computation remains. recurrent models typically factor computation along the symbol positions of the input and output sequences. tensor2tensor i

In [52]:
# Final summarization of the concatenated summary
final_summary_inputs = tokenizer(summarized_text, return_tensors="pt", truncation=True, padding="longest")
final_summary_ids = model.generate(final_summary_inputs.input_ids, max_length=1000, min_length=300, do_sample=False)
final_summary = tokenizer.decode(final_summary_ids[0], skip_special_tokens=True)



In [53]:
# Print the final refined summary
print("Final Refined Summary:")
print(final_summary)

Final Refined Summary:
BLEU score of 41.0 after training for 3.5 days on eight GPUs. Our model establishes a new single-model state-of-the-art BLEU score of 41.0 after training for 3.5 days on eight GPUs. tensor2tensor is a recurrent language model, and is a key component of.....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## Another approach.

In [54]:

# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [56]:
# max tokens including the special tokens
tokenizer.model_max_length



1024

In [58]:
# Convert file content to sentences

# extract the sentences from the document
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

291

In [61]:
# Create the chunks

# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

9

In [63]:
# Some checks

[len(tokenizer(c).input_ids) for c in chunks]

[1010, 1007, 995, 1014, 1006, 1005, 1011, 1004, 422]

In [64]:
# With special tokens added
sum([len(tokenizer(c).input_ids) for c in chunks])

8474

In [66]:
len(tokenizer(text).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (8557 > 1024). Running this sequence through the model will result in indexing errors


8557

In [67]:
# Without special tokens added


sum([len(tokenizer.tokenize(c)) for c in chunks])

8456

In [69]:
len(tokenizer.tokenize(text))

8555

In [70]:
# Get the inputs

# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [71]:
# Get the summarized text.


for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

 We propose a new simple network architecture, the Transformer, based solely on attention mechanisms. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring signiﬁcantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task.
 The Transformer is the ﬁrst transduction model relying solely on self-attention to compute representations of its input and output without using sequence-aligned RNNs or convolution. The model is based on a recurrent attention mechanism instead of sequence-related recurrence. We use stacked self-association layers for both the encoder and decoder, shown in the left and right halves of Figure 1.
 Multi-Head Attention consists of several parallel attention layers running in parallel. We compute the attention function on a set of queries simultaneously, packed together into a matrix Q. Additive attention computes the compatibility function usi